## <center> École Polytechnique de Montréal <br> Département Génie Informatique et Génie Logiciel <br>  LOG6308 - Systèmes de recommandations <br> </center>

## <center> TP2 -- Approche collaboratives par factorisation et par agglomération, et approche contenu </center>

Le travail doit être fait en **équipe de deux**. 

## Identification de l'équipe: \<Groupe\>_eq\<Numero_Equipe\>

### Groupe de laboratoire: ...

### Equipe numéro : ...

### Membres:

Etudiant 1 (Matricule)

Etudiant 2 (Matricule)

<br>

**Nature de la contribution:** Décrivez brièvement ce qui a été fait par chaque membre de l’équipe. Tous les membres sont censés contribuer au développement. Bien que chaque membre puisse effectuer différentes tâches, vous devez vous efforcer d’obtenir une répartition égale du travail. Soyez précis sur la contribution de chacun.

## Enoncé du TP

### Introduction
Après avoir exploré les bases des systèmes de recommandation dans le TP1, notamment à travers des approches utilisateur-utilisateur et item-item de filtrage collaboratif, ce deuxième TP aborde les approches basées sur la **factorisation matricielle (SVD)** et le **clustering (K-means)**. Ce TP aborde également des concepts clés tels que la réduction de dimensions, la sélection du nombre optimal de classes, et l’utilisation de catégories pour prédire les votes d’un nouvel utilisateur.

À travers ces méthodes, vous serez amenés à analyser les avantages et limites des approches proposées, tout en consolidant votre compréhension des techniques utilisées dans les systèmes de recommandation modernes. Comme dans le TP1, des validations croisées seront mises en œuvre pour évaluer la robustesse des algorithmes. N’hésitez pas à poser des questions si des points restent flous. 

Bon travail !


### Contexte général
Vous travaillez avec un jeu de données contenant des votes d’utilisateurs sur des films. Certains votes manquent : votre objectif est de prédire les votes manquants pour chaque utilisateur/film.

Pour évaluer vos prédictions, vous utiliserez deux métriques :

- Erreur quadratique moyenne (RMSE)
- Erreur absolue moyenne (MAE)


### Jeux de données

Vous avez 3 fichiers à votre disposition:

- 'Data/votes.csv': Matrice de données de 100 000 votes faits par 943 utilisateurs et portant sur 1682 items.
    - **user.id**: Indentifiant de l'utilisateur
    - **item.id**: Identifiant de l'item/film
    - **rating**: vote attribué à l'item par l'utilisateur
    - **timestamp**: Date d'enregistrement du vote (à ignorer pour ce TP) 
- 'Data/items.csv': Matrice de données sur les films
    - **movie.id**: Identifiant du film
    - **movie.title**: Nom du film
    - **release.date**: Date de sortie
    - **video.release.date**: Date de sortie de la video
    - **IMDb.URL**: Lien vers le film
    - les 19 autres champs sont les categories des films qui sont les suivantes:
        "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
- 'Data/u.csv': Matrice de données sur les utilisateurs
    - **id**: Identififiant de l'utilisateurs
    - **age**: Age de l'utilisateur
    - **gender**: Sexe de l'utilisateur
    - **job**: Emploi de l'utilisateur
    - **zip**: ZIP Code

Attention aux espaces et à la casse des differents champs. 

### Librairies permises

- numpy
- pandas
- seaborn
- matplotlib
- nltk (KMeansClusterer)
- scipy (stat)
- tqdm

### Rédaction et remise du rapport

- Ce notebook constitue à la fois votre code et votre rapport. Il contient un squelette pour guider votre travail.

- Complétez directement le notebook, vous êtes libres de créer des nouvelles cellules de code ou de texte.

- <u>**IMPORTANT**</u> Remettez le ZIP contenant les données et le notebook sur Moodle avec le nom `MATRICULE1_MATRICULE2.ipynb` pour le notebook et `MATRICULE1_MATRICULE2.zip` pour le zip.


### CRITÈRES

- La démarche est valide et bien expliquée
- Les réponses sont correctes et commentées
- L'implémentation est performante et repose sur le calcul linéaire lorsqu'approprié
- La présentation est soignée et bien rédigée


### CODE D’HONNEUR

- __Règle 1__:  Le plagiat de code est bien évidemment interdit. Toute utilisation de code doit être référencée adéquatement. Vous __ne pouvez pas__ soumettre un code écrit par quelqu’un d’autre.

- __Règle 2__: Vous êtes libres de discuter des idées et des détails de mise en œuvre avec d'autres équipes. Cependant, vous ne pouvez en aucun cas consulter le code d'une autre équipe ou incorporer leur code dans votre TP.

- __Règle 3__:  Vous ne pouvez pas partager votre code publiquement (par exemple, dans un dépôt GitHub public) tant que le cours n'est pas fini.

In [9]:
import numpy as np
import pandas as pd 
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import nltk


from nltk.cluster.kmeans import KMeansClusterer
from scipy.stats import pearsonr
from tqdm import tqdm

## Question 1 : Factorisation par Valeurs Singulières (SVD) (12 points)

La factorisation matricielle, via la décomposition SVD, est une méthode puissante pour capturer les relations latentes dans les données. Dans le cadre de ce TP, elle sert à :

- Réduire la matrice des votes en identifiant les facteurs latents qui expliquent les interactions utilisateur-item.
- Identifier les dimensions les plus significatives (les relations principales) et réduire le bruit lié à des données éparses ou des votes erronés.
- Améliorer la qualité des prédictions en reconstruisant les valeurs manquantes sur la base des facteurs latents, tout en diminuant la complexité computationnelle.

L’objectif est donc de démontrer comment la réduction de dimensions peut simplifier et impacter la précision des systèmes de recommandation, tout en explorant le compromis entre complexité et performance.

**Instructions**:
Utilisez l'approche de factorisation par valeurs singulières (SVD) et calculez l'erreur quadratique moyenne. Déterminez le nombre de dimensions à retenir par une méthode de votre choix. Effectuez une validation croisée de 5 replis.

**Définition des Métriques**

In [ ]:
def RMSE_mat(y_pred, y_true):
    return np.sqrt(np.nanmean((y_pred - y_true)**2))

def MAE_mat(y_pred, y_true):
    return np.nanmean(np.abs(y_pred - y_true))


**Préparation des Données**

Pensez a changer le separateur de "|" vers "," ou reciproquement si vous avez un problemes de chargement de données.

In [ ]:
## Chargement des votes
votes = pd.read_csv('Data/votes.csv', sep =',')# Enlever le sep='|' pour vos données

## Conversion du Pandas Datafram en Matrice Utilisateur Item
MUI = votes.pivot(index="user.id", columns="item.id", values="rating")
MUI.head()


## Convertir le DF à une matrice numpy
MUI_numpy      = MUI.to_numpy()
MUI_numpy_flat = MUI_numpy.reshape(-1)

In [ ]:
# Charger les autres données en mémoire
# items 
items = pd.read_csv('Data/items.csv', sep=',')
# users
user = pd.read_csv('Data/u.csv', sep=',')
# jobs matrices
jobs = pd.read_csv('Data/jobs-matrix.csv', sep=',', index_col=0)

In [ ]:
## Defining Figure Size:
sns.set(rc={"figure.figsize":(12, 6)})

In [ ]:
##Pour la Validation croisée
## Création des indices pour les valeurs différentes de np.nan
indices    = np.arange(0, MUI_numpy.shape[0]*MUI_numpy.shape[1])
indices_na = indices[~np.isnan(MUI_numpy_flat)]

## Split Train Test des indices
nbre_replis = 5
np.random.shuffle(indices_na)
idx_split = np.split(indices_na, nbre_replis)

Completez la fonction ci-dessous afin de calculer le biais.  Ce calcul permettra de faire l'imputation des valeurs manquantes.

In [ ]:
## Calcul de la moyenne attendue

def Biais_mat(R):

    # Vote Moyen Utilisateur
    user_mean = ?

    # Prédiction des votes grâce à la moyenne des votes par item de la matrice d'entrainement
    item_mean = ?

    #calculer la moyenne attendue pour utilisateur
    moyenne_U_repeat = ?

    #calculer le moyenne attendue pour item
    moyenne_I_repeat = ?

    # Notre Baseline: vote moyen attendu
    R_moy = ?

    return R_moy

**Définition des Fonctions pour l'approche SVD**

L'objectif est de faire une decomposition matricielle suivant l'approche SVD. Puis une reduction de dimension en selectionnant un sous ensemble de valeur singulieres (les $k$ plus grandes); et finalement faire une prediction de votes. 

Completez les fonction suivantes pour effectuer ces tâches. 

Utilisez la décomposition SVD :   $ R = U \Sigma V^T $  
  
Où :  
- $M$ est la matrice des votes.  
- $ U $ et $ V $ sont des matrices représentant les utilisateurs et les items.  
- $ \Sigma $ est une matrice diagonale contenant les valeurs singulières.  


In [ ]:
def SVD_decomp(R):
    u, sigma, v_trans = ?
    #transformation du vecter sigma en une matrice diagonale :
    sigma = np.diag(sigma) 

    return u, sigma, v_trans

def SVD_matpred(u, sigma, v_trans, k=5):
    #faire une matrice de prédiction en refaisant la multiplication matricielle de u,
    #sigma et v_trans mais en utilisant uniquement les k dimensions latentes :
    # [compléter le code]
    prediction = ?

    return prediction

**Prédiction de votes et calculs d'erreurs**  

Maintenant l'objectif est de faire les predictions de votes en fonction du nombre de valeurs singulières selectionné et de calculer les erreurs RMSE et MAE.

In [ ]:
list_k = [2, 5, 10, 15, 20, 30, 40, 50, 100, 200, 400]

**Sans validation croisée**

In [ ]:
# Decomposition
list_RMSE_svc = []
list_MAE_svc  = []

R_biais    = ?
R_centered = ?
R_centered[np.isnan(R_centered)] = 0

u, sigma, v_trans = ?
for k in list_k:
    # Pred
    R_pred = ?

    list_RMSE_svc.append(RMSE_mat(R_pred + ?, MUI_numpy))
    list_MAE_svc.append(MAE_mat(R_pred + ?, MUI_numpy))

In [ ]:
# Afficher la courbe RMSE en fonction du nombre de dimensions.
?

In [ ]:
# Déterminer la meilleure valeur de K
?

**Validation croisée**

Indication: 
- On itère sur les replis avant d'itérer sur le nombre de valeurs singulières $k$ que l'on retient. C'est un détail qui permet d'être plus performant en temps de calcul.
- Un variable contenant les indices a déjà été crée plus haut

In [ ]:
%%time
list_RMSE = []
list_MAE  = []

for i in range(nbre_replis):
    list_RMSE_i = []
    list_MAE_i  = []

    idx_train = ?
    idx_valid = ?
    
    ## On enlève les valeurs de test de la matrice d'entrainement, et les valeurs de train dans la matrice de test
    

    ## On redonne la structure de matrice aux ensembles de test et d'entrainement
    R_train = ? # Valeurs de train
    MUI_numpy_valid = ? # Valeurs de test

    # Centré
    R_biais    = ?
    R_centered = ?
    R_centered[np.isnan(R_centered)] = 0

    # Decomposition
    u, sigma, v_trans = ?
    for k in list_k:
        # Pred
        R_pred = ?

        list_RMSE_i.append(RMSE_mat(R_pred + ?, MUI_numpy_valid))
        list_MAE_i.append(MAE_mat(R_pred + ?, MUI_numpy_valid))

    list_RMSE.append(np.array(list_RMSE_i))
    list_MAE.append(np.array(list_MAE_i))

list_RMSE = np.array(list_RMSE)
list_MAE  = np.array(list_MAE)

In [ ]:
# Mean per value of k
list_RMSE_mean = list_RMSE.mean(axis=0)

In [ ]:
values_of_k    = [k for i in range(nbre_replis) for k in list_k]
values_of_rmse = list_RMSE.flatten()
df_q1 = pd.DataFrame.from_dict({"Number of Dimensions Taken": values_of_k, "Root Mean Square Error on Validation": values_of_rmse})
df_q1.sort_values("Number of Dimensions Taken").head()

In [2]:
# Afficher les courbes de RMSE en fonction du nombre de dimensions choisies
?

In [3]:
# Déterminer la meilleure valeur de K et comparé vos resultats avec ceux obtenus dans l'approches sans validation croisée.
?

**Prédiction**

Supposons un nouvel utilisateur qui a visionné le film "Clockwork Orange" (numéro 179, indice 178 si les indices commencent à 0).  Utilisez l'approche SVD de prédiction pour lui recommander 10 films.

In [ ]:
# Fonction qui prend en argument un vecteur ou une matrice de votes d'un utilisateur (lignes de la matrice) et retourne les indices des k films les plus similaires
def k_most_similar(votes, k=10):
    # à compléter

## Question 2 (8 points)

L’algorithme K-means est une méthode de regroupement/agglomération (*clustering*) qui divise les utilisateurs (ou items) en plusieurs classes homogènes basées sur leurs similarités. Dans le cadre de ce TP, K-means sert à :

- Identifier des groupes d’utilisateurs ou d’items similaires selon leurs interactions (par exemple, des utilisateurs ayant des préférences proches ou des films avec des profils similaires de spectateurs).
- Simplifier les prédictions en supposant que les membres d’un même groupe auront des comportements similaires (par exemple, des utilisateurs d’un même cluster auront des votes comparables).
- Tester comment le choix du nombre de classes ($k$) influence la précision des prédictions (RMSE) et la capacité du modèle à capturer les relations dans les données.

Cette approche permet d’explorer comment le regroupement peut simplifier la personnalisation des recommandations et servir d’alternative (ou de complément) à d'autres méthodes comme SVD ou les filtres collaboratifs.

**Instructions :**
  
Utilisez une approche par agglomération (clustering) pour prédire les votes et calculez l'erreur quadratique moyenne. Utilisez 2, 5, 10, 20, et 40 classes. Prenez l'approche K-moyenne (K-means) et utilisez la corrélation comme mesure de distance et une validation croisée de 5 replis pour établir quel est le nombre de classes à retenir parmi les 5 mentionnés. Une bonne explication de l'algorithme k-means est fournie par Alexandre Ihler dans cette [vidéo](https://www.youtube.com/watch?v=mfqmoUN-Cuw&feature=youtu.be).

### Distances
Afin d'effectuer la mesure de distance dans l'approches KMEANS, les fonctions suivantes calculent la distance de Pearson et la distance Euclidienne.

#### Distance de Pearson

In [ ]:
## Distance de Pearson avec des Nans

def PearsonDist(x, y):
    if(len(x.shape) < len(y.shape)):# For Cluster Assignment
        x = np.expand_dims(x, axis=0).repeat(y.shape[0], axis=0)
        nan_or = np.logical_or(np.isnan(x), np.isnan(y))
        corr   = [pearsonr(x[i, ~nan_or[i]], y[i, ~nan_or[i]])[0] if((~nan_or[i]).sum() >=2) else 1 for i in range(y.shape[0])]
    elif(len(x.shape) > len(y.shape)):
        y = np.expand_dims(y, axis=0).repeat(x.shape[0], axis=0)
        nan_or = np.logical_or(np.isnan(x), np.isnan(y))
        corr   = [pearsonr(x[i, ~nan_or[i]], y[i, ~nan_or[i]])[0] if((~nan_or[i]).sum() >=2) else 1 for i in range(y.shape[0])]
    else:# For training phase
        nan_or = np.logical_or(np.isnan(x), np.isnan(y))
        if((~nan_or).sum() <2):
            return 1
        corr   = pearsonr(x[~nan_or], y[~nan_or])[0]
    return 1 - np.abs(corr)

#### Distance Euclidienne

In [ ]:
def EuclidDist(x, y):
    # x, y = np.array(x), np.array(y)
    nan_or = np.logical_or(np.isnan(x), np.isnan(y))
    if((~nan_or).sum() <2):
        return nan_or.shape[0]
    return np.linalg.norm(x[~nan_or] - y[~nan_or])

In [ ]:
%%time

list_RMSE_q2 = []
list_MAE_q2  = []
list_k_q2    = [2, 5, 10, 20, 40]

for i in range(nbre_replis):
    list_RMSE_i = []
    list_MAE_i  = []

    idx_train = ?
    idx_valid = ?

    # On enlève les valeurs de test de la matrice d'entrainement, et vice versa


    # On redonne la structure de matrice aux ensembles de test et d'entrainement
    MUI_numpy_train = MUI_numpy_flat_train.reshape(MUI_numpy.shape)
    MUI_numpy_valid = MUI_numpy_flat_valid.reshape(MUI_numpy.shape)


    for num_cluster in list_k_q2:
        ## Remplacer les Nans par le vote moyen pour que K-Means converge en utilisant la fonction Biais_mat définie plus haut.
        data                 = MUI_numpy_train.copy()
        data[np.isnan(data)] = ?

        ## Entrainement du K-Means
        kclusterer        = ?
        assigned_clusters = ?

        ## Extraction des centroides
        centroids         = np.array([np.nanmean(MUI_numpy_train[assigned_clusters==k], axis=0) for k in range(num_cluster)])

        ## Matrice de prédictions
        assigned_clusters_valid = ?
        R_pred                  = ?

        list_RMSE_i.append(RMSE_mat(R_pred, MUI_numpy_valid))
        list_MAE_i.append(MAE_mat(R_pred, MUI_numpy_valid))

    list_RMSE_q2.append(np.array(list_RMSE_i))
    list_MAE_q2.append(np.array(list_MAE_i))

list_RMSE_q2 = np.array(list_RMSE_q2)
list_MAE_q2  = np.array(list_MAE_q2)

In [ ]:
values_of_k_q2    = [k for i in range(nbre_replis) for k in list_k_q2]
values_of_rmse_q2 = list_RMSE_q2.flatten()
df_q2 = pd.DataFrame.from_dict({"Number of Dimensions Taken": values_of_k_q2, "Root Mean Square Error on Validation": values_of_rmse_q2})
df_q2.sort_values("Number of Dimensions Taken").head()

In [4]:
# Afficher la courbe RMSE en fonction du nombre de dimensions.

In [ ]:
# Conclure